In [1]:
import pandas as pd
from contentTFIDF import ContentTFIDF

data = pd.read_csv('./global_track_spotify.csv', encoding = 'utf-8')
ctfidf = ContentTFIDF(data)
ctfidf.preprocess()
ctfidf.saveTFIDF()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yonjae/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/yonjae/opt/anaconda3/envs/spotify/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
import os
import re
import numpy as np
import pandas as pd
import argparse
import mlflow
import mlflow.pyfunc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore')


class ContentBasedRecommenderSystem:
    
    def __init__(self, data, tfidf, music, mood, speed, emotion):
        self.data = data
        self.tfidf = tfidf
        self.music = music
        self.mood = mood
        self.speed = speed
        self.emotion = emotion
        self.result = pd.DataFrame()

    def cleanText(self, text_data):
        text = re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text_data)
        return text

    def preprocess(self):
        genre = []
        for i in self.data['artist_genre']:
            if i == '[]':
                i = 'NA'
                genre.append(i.strip()) #"'[]'"
            else:
                i = self.cleanText(i)
                genre.append(i.strip())
        self.data['genre'] = genre
        self.data = self.data[self.data['genre'] != "NA"]
        self.data = self.data.reset_index()
        self.data['track_popularity'] = self.data['track_popularity'] / 100 

    def euclidean_distance(self, x, y):   
        return np.sqrt(np.sum((x - y) ** 2))
    
    def recommend_features(self, top=200):
        scaler = MinMaxScaler()
        index = self.data[self.data['track_name'] == self.music].index.values
        track_new = self.data[['danceability', 'energy', 'valence', 'tempo', 'acousticness']]
        track_scaled = scaler.fit_transform(track_new)
        target_index = track_scaled[index]

        euclidean = []
        for value in track_scaled:
            eu = self.euclidean_distance(target_index, value)
            euclidean.append(eu)

        self.data['euclidean_distance'] = euclidean
        result_features = self.data.sort_values(by='euclidean_distance', ascending=True)[:top]

        return result_features[['id','artist_name', 'track_name', 'euclidean_distance']]

    
    def recommend_genre(self, top=200):
        # 코사인 유사도
        ts_genre = cosine_similarity(self.tfidf, self.tfidf)

        #특정 장르 정보 뽑아오기
        target_genre_index = self.data[self.data['track_name'] == self.music].index.values

        # 입력한 영화의 유사도 데이터 프레임 추가
        self.data["cos_similarity"] = ts_genre[target_genre_index, :].reshape(-1,1)
        sim_genre = self.data.sort_values(by="cos_similarity", ascending=False)
        final_index = sim_genre.index.values[ : top]
        result_genre = self.data.iloc[final_index]

        return result_genre[['id','artist_name', 'track_name', 'cos_similarity']]

    
    def get_feature_genre_intersection(self):
        recommended_feature = self.recommend_features()
        recommended_genre = self.recommend_genre()
        intersection = pd.merge(recommended_feature, recommended_genre, how='inner')
        similarity = intersection[['euclidean_distance', 'cos_similarity']]
        scaler = MinMaxScaler()
        scale = scaler.fit_transform(similarity)
        scale = pd.DataFrame(scale, columns=['eu_scaled', 'cos_scaled'])
        
        intersection['euclidean_scaled'] = scale['eu_scaled']
        intersection['cosine_scaled'] = scale['cos_scaled']
        intersection['ratio'] = intersection['euclidean_scaled'] + (1 - intersection['cosine_scaled'])
        result_intersection = intersection.sort_values('ratio', ascending=True)
        self.result = pd.merge(self.data, result_intersection, how='inner').sort_values(by='ratio')
        
        return self.result

    
    def get_genre_score(self):
        cosine_sim_score = cosine_similarity(self.tfidf, self.tfidf)
        target_genre_index = self.result[self.result['track_name'] == self.music].index.values
        genre_score = cosine_sim_score[target_genre_index, :].reshape(-1, 1)
        return genre_score

    
    def get_mood_score(self):
        temp = pd.DataFrame(self.result['valence'])
        if self.mood == 1:
            temp['mood_score'] = temp['valence']
        else:
            temp['mood_score'] = temp['valence'].apply(lambda x: 1-x)
        return temp['mood_score']
    
    
    def get_speed_score(self):
        temp = pd.DataFrame(self.result['tempo'])
        temp['tempo_scaled'] = MinMaxScaler().fit_transform(pd.DataFrame(temp['tempo']))
        if self.speed == 1:
            temp['speed_score'] = temp['tempo_scaled']
        else:
            temp['speed_score'] = temp['tempo_scaled'].apply(lambda x: 1-x)
        return temp['speed_score']

    
    def get_emotion_score(self):
        temp = self.result[['danceability', 'energy', 'acousticness']]
        temp['danceability_scaled'] = MinMaxScaler().fit_transform((pd.DataFrame(temp['danceability'])))
        temp['acousticness_reverse'] = temp['acousticness'].apply(lambda x: 1-x)
        if self.emotion == 1:
            temp['emotion_score'] = temp.apply(lambda x: 1/3 * (x['danceability_scaled'] + x['energy'] + x['acousticness_reverse']), axis = 1)
        elif self.emotion == 2:
            temp['emotion_score'] = temp.apply(lambda x: 2/3 * (abs(x['danceability_scaled']-0.5) + abs(x['energy']-0.5) + abs(x['acousticness_reverse']-0.5)), axis = 1)
        else:
            temp['emotion_score'] = temp.apply(lambda x: 1/3 * ((1-x['danceability_scaled']) + (1-x['energy']) + (1-x['acousticness_reverse'])), axis = 1)
        return temp['emotion_score']

    def get_total_score(self, top_n = 20):
        result_df = self.result[['artist_name', 'track_name', 'album_name']]
        result_df['mood_score'] = pd.DataFrame(self.get_mood_score())
        result_df['speed_score'] = pd.DataFrame(self.get_speed_score())
        result_df['emotion_score'] = pd.DataFrame(self.get_emotion_score())
        result_df['genre_score'] = pd.DataFrame(self.get_genre_score())
        result_df['total_score'] = result_df.apply(lambda x: 1/6*(x['mood_score'] + x['speed_score'] + x['emotion_score']) + 0.5*x['genre_score'], axis = 1)
        
        return result_df.iloc[1:].sort_values(by = 'total_score', ascending=False)[:top_n]


track = pd.read_csv('./data/track/track_data.csv', encoding = 'utf-8')
ct_tfidf = pd.read_csv('./data/tfidf/tfidf_matrix.csv', encoding = 'utf-8')
    
cbr = ContentBasedRecommenderSystem(track, ct_tfidf, 'Ring My Bell', 1, 1, 1)
cbr.preprocess()
cbr.get_feature_genre_intersection()
total_score = cbr.get_total_score()

AttributeError: 'ContentBasedRecommenderSystem' object has no attribute 'tfidf'

In [3]:
total_score

,artist_name,track_name,album_name,mood_score,speed_score,emotion_score,genre_score,total_score
15,TAEYEON,Rain,Rain - SM STATION,0.594,0.907885,0.216941,1.000000,0.786471
26,TAEYEON,Galaxy,What Do I Call You - The 4th Mini Album,0.660,0.815451,0.221360,0.417855,0.491729
23,D.O.,My Love,공감 (Empathy) - The 1st Mini Album,0.653,0.735545,0.376463,0.182481,0.385409
18,SE7EN,Come Back To Me,Just Listen.......,0.204,0.221189,0.364371,0.484197,0.373692
14,BOL4,Hard To Love 나만 안되는 연애,Full Album RED PLANET,0.625,1.000000,0.409553,0.000000,0.339092
11,BIGBANG,LOSER,M,0.512,0.737042,0.237943,0.139104,0.317383
3,IU,Friday (feat.Jang Yi-jeong),Modern Times – Epilogue,0.449,0.858459,0.137277,0.136502,0.309040
13,WINNER,AH YEAH,WE,0.122,0.269989,0.412676,0.333433,0.300827
4,IU,Meaning of you,A flower bookmark,0.671,0.861220,0.252273,0.000000,0.297415
25,TAEMIN,My Day,"Navillera (Original Television Soundtrack), Pt. 1",0.528,0.858901,0.349853,0.000000,0.289459
